# Quantum singular value estimation

Ryan LaRose

### Abstract

Singular value decomposition is a powerful linear algebra technique. A quantum algorithm for estimating singular values, called quantum singular value estimation (QSVE), was presented in [1] for the purpose of recommendation systems. Later, it was shown how QSVE could be used as a subroutine for solving linear systems of equations [2]. In this notebook, we demonstrate how to use the Qiskit Aqua implementation of QSVE and explain how the algorithm works. In other notebooks, we present the applications to recommendation systems and linear systems in Qiskit Aqua.

In [ ]:
"""Imports for the notebook."""
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse

import qiskit
from qiskit.aqua.components.qsve import QSVE, BinaryTree
from qiskit.aqua.utils import random_hermitian

# Introduction

Although not all matrices can be diagonalized, every matrix has a singular value decomposition. If $A$ is an $m \times n$ matrix with complex entries, we can write

\begin{equation}
    A = U \Sigma V^\dagger = \sum_i \sigma_i \mathbf{u}_i \mathbf{v}_i^\dagger
\end{equation}

where $U \in \mathbb{C}^{m \times m}$ is unitary, $\Sigma \in \mathbb{R}^{m \times n}$ has non-negative diagonal components, and $V \in \mathbb{C}^{n \times n}$ is unitary. The above equation can equivalently be written

\begin{equation}
    A = \sum_i \sigma_i \mathbf{u}_i \mathbf{v}_i^\dagger
\end{equation}

where $\sigma_i \ge 0$ are the diagonal elements (called _singular values_) of $\Sigma$, and $\mathbf{u}_i$ ($\mathbf{v}_i$) is the $i$th column vector (called _singular vectors_) of $U$ ($V$).

# Examples and usage

In this section, we present examples of how to use QSVE in Qiskit Aqua. We assume the input matrix is Hermitian, which can always be done. If this is not the case, the user is responsible for embedding the matrix into a Hermitian system, for example via the transformation

\begin{equation}
    A \longmapsto \left[ \begin{matrix}
                0 & A \\
                A^\dagger & 0 \\
                \end{matrix} \right] .
\end{equation}

For the simple examples in this notebook, we ensure Hermiticity by generating a matrix $A$ then transforming via $A \longmapsto A + A^\dagger$. 

### Example 1: Identity Matrix

In [ ]:
"""Example of computing the singular values of the identity matrix."""
# Define the matrix of the system
matrix = np.identity(2)

# Get a QSVE instance
qsve = QSVE(matrix)

# Estimate and return the top singular values
qsigmas = qsve.top_singular_values()

# Compute the singular values classically for comparison
sigmas = qsve.singular_values_classical()

# Display the singular values
print("The singular values from QSVE are:", qsigmas)
print("The singular values from classical SVD are:", sigmas)

In [ ]:
"""Looking at the circuit and measurement outcomes of QSVE."""
# Create the circuit and run it to get the measurement outcomes
circuit = qsve.create_circuit(nprecision_bits=1, logical_barriers=True)
counts = qsve.run_and_return_counts()

# Display the circuit
print(circuit)
print("\nCounts =", counts)

In [ ]:
"""Converting bit strings to angles and singular values."""
# Measured bit strings from the above example
bitstring1 = "011"
bitstring2 = "010"

print("Measured bitstrings:")
print("bitstring1 =", bitstring1)
print("bitstring2 =", bitstring2)

# Convert these measured strings to floating point values
float1 = qsve.binary_decimal_to_float(bitstring1)
float2 = qsve.binary_decimal_to_float(bitstring2)

# Put the floating point angles in the interval [-1/2, 1/2]
theta1 = qsve.convert_measured(float1)
theta2 = qsve.convert_measured(float2)

print("\nBitstrings converted to angles in [-1/2, 1/2]:")
print("theta1 =", theta1)
print("theta2 =", theta2)

# Convert the angles to singular values
qsigma1 = qsve.angle_to_singular_value(theta1)
qsigma2 = qsve.angle_to_singular_value(theta2)

print("\nSingular values found via QSVE:")
print("qsigma1 = ", qsigma1)
print("qsigma2 = ", qsigma2)

Since the measured bit strings of this unitary can be exactly expressed with three bits, we should not see any change as we go to $n > 3$ qubits for QPE. This is shown in the following cell.

In [ ]:
"""Increasing the number of QPE qubits and checking the bit strings."""
for n in range(3, 7):
    counts = qsve.run_and_return_counts(nprecision_bits=n)
    print("Number of qubits = {}, counts = {}".format(n, counts))

As can be seen, the bit strings do not change with more QPE qubits.

__Note__: The bit strings are "little endian," which means that the most significant bit is the last in the bit string.

### Example 2: Convergence towards singular values

In this example, we show how increasing the number of qubits used in QPE increases the accuracy of the singular value estimates. We first create a random two by two matrix, then use QSVE to estimate the top eigenvalue.

In [ ]:
"""Example of convergence toward singular values."""
# Get a random matrix (seed for reproducibility)
np.random.seed(402457087)
matrix = np.random.randn(2, 2)
matrix += matrix.conj().T

# matrix = random_hermitian(2, eigs=[0.4, 0.2])

# Get a QSVE instance and compute the singular values classically
qsve = QSVE(matrix)
sigmas = qsve.singular_values_classical()

print("Classical singular values:", np.round(sigmas, 5))
print("============================================")

nqubits = list(range(1, 8 + 1))
qsigmas = []
errors = []
theoretical_errors = []

for n in nqubits:
    print("Status: n =", n)
    qsigmas += qsve.top_singular_values(nprecision_bits=n, ntop=1)
    errors.append(abs(qsigmas[-1] - sigmas[0]))
    theoretical_errors.append(qsve.max_error(n))
    print("Current estimate:", np.round(qsigmas[-1], 5))

In [ ]:
"""Plot the convergence to the singular value in the number of qubits for QPE."""
# Figure out which singular value was more prominent in the measurement statistics
if np.isclose(qsigmas[-1], sigmas[0], atol=1e-2):
    ind = 0
else:
    ind = 1

plt.rcParams.update({"font.size": 16, "font.family": "serif", "font.weight": "bold"})

fig = plt.figure(figsize=(16, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.plot(nqubits, sigmas[ind] * np.ones(len(nqubits)), "-", linewidth=4, label="Analytic");
ax1.plot(nqubits, qsigmas, "--o", linewidth=4, markersize=12, label="QSVE");
ax1.set_xlabel("Number of qubits in QPE");
ax1.set_ylabel("Normalized singular value");
ax1.set_title("Convergence towards singular value");
ax1.grid();
ax1.legend();

ax2.plot(nqubits, theoretical_errors, "-", linewidth=4, label="Max theoretical error");
ax2.plot(nqubits, errors, "--o", linewidth=4, markersize=12, label="Numerical error");
ax2.set_xlabel("Number of qubits in QPE");
ax2.set_ylabel("Error");
ax2.set_title("Error convergence towards zero");
ax2.grid();
ax2.legend();

__Note__: The error plot assumes the top singular value is the largest one (i.e., the first in `sigmas`), which may not always be the case. If you change the matrix and the error does not seem to be going to zero, this is the reason. Simply change the `sigmas[0]` to `sigmas[1]` in the previous cell where the errors are computed to fix this.

Note that the simulation time quickly increases in the number of QPE qubits. For this example, going beyond eight qubits requires long times to execute.

### Example 3: A larger matrix

Here we implement QSVE on an eight by eight sparse matrix. Below, we will see that the number of gates in the algorithm scales with the number of nonzero matrix elements, so we consider a sparse matrix here for efficiency.

In [ ]:
"""Example: QSVE on a random sparse matrix."""
# Get a matrix and display it
sp = sparse.random(4, 4, density=0.3)
matrix = sp.toarray()
matrix += matrix.conj().T
print("Matrix:","=======", np.round(matrix, 3), sep="\n")

# Do the classical SVD and quantum SVE
qsve = QSVE(matrix)
sigmas = qsve.singular_values_classical()
print("\nClassically found singular values:", np.round(sigmas, 5))
qsigmas = qsve.top_singular_values(nprecision_bits=5, ntop=4)
print("\nQuantum singular value estimates:", np.round(qsigmas, 5))

Here, you should see that the quantum singular value estimatesare close to one or more singular values. You may also notice this example takes a bit longer to run, and we can understand why by inspecting the circuit below.

In [ ]:
"""Example: Checking the number of gates in the circuit."""
circuit = qsve.create_circuit(nprecision_bits=5)
native = circuit.decompose()
print(native.count_ops())

You should see that this circuit has on the order of 10,000 CNOT gates and a comparable number of single qubit gates. These numbers quickly reach the capabilities of the simulators, and are far beyond the state of current quantum computers.

# How the algorithm works

The QSVE algorithm uses quantum phase estimation (QPE) on a particular unitary $W$, the eigenvalues of which are related to the singular values of the given matrix $A$. (Recall that QPE estimates eigenvalues of a unitary matrix.)

## Defining the unitary for QPE

To be concrete, let $A \in \mathbb{R}^{n \times n}$ be Hermitian with Froebenius norm $||A||_F$. To build up the unitary $W \in \mathbb{R}^{n^2 \times n^2}$, we define the isometry $M \in \mathbb{R}^{n^2 \times n}$ as

\begin{equation}
    M := 
\end{equation}

and the isometry $N \in \mathbb{R}^{n \times n^2}$ as

\begin{equation}
    N := 
\end{equation}

The unitary $W$ is then defined as

\begin{equation}
    W := (2 MM^\dagger - I)(2N N^\dagger - I)
\end{equation}

Note that $I$ denotes the identity operator and both terms in parentheses are reflection operators. 

It can be shown that the eigenvalues of $W$ are related to the singular values of $A$. In particular, suppose $e^{i \pi \theta_j}$ is an eigenvalue of $W$ where $\theta_j \in [-1, 1]$. Then, 

\begin{equation}
    \cos \pi \theta_j = \frac{\sigma_j}{||A||_F}
\end{equation}

Using QPE to compute the bit strings $\theta_j$ thus allows us to estimate the singular values of the desired matrix $A$. 

## Building the unitary in a quantum circuit

### Loading a real vector onto a quantum state using a BinaryTree

In [ ]:
"""Example of using a BinaryTree to load amplitudes onto a quantum state."""
# Define a matrix
matrix = np.array([[0.6, 0.8],
                   [0.8, 0.6]])

# Get the first row vector of the matrix
row = matrix[0]

# Make a BinaryTree out of the row vector and display it
tree = BinaryTree(row)
print(tree)

In [ ]:
"""Continued: Example of using a BinaryTree to load amplitudes onto a quantum state."""
# Define a quantum register and quantum circuit
qreg = qiskit.QuantumRegister(1)
circ = qiskit.QuantumCircuit(qreg)

tree.preparation_circuit(circ, qreg)
print(circ)

In [ ]:
"""Continued: Example of using a BinaryTree to load amplitudes onto a quantum state."""
# Make sure the final state of the circuit is the initial vector (normalized)
simulator = qiskit.BasicAer.get_backend("unitary_simulator")
job = qiskit.execute(circ, simulator)

unitary = np.array(job.result().results[0].data.unitary)
state = np.real(unitary[:, 0])

print(f"Prepared state: {state}")

The `BinaryTree` works for loading a vector of any size (granted its a power of two) with positive or negative components, but they must be real valued. The number of gates scales with the number of non-zero vector components. An example of loading a four element vector onto two qubits is shown below.

In [ ]:
"""Example of loading a vector of four elements using a BinaryTree."""
vector = np.random.randn(4)
tree = BinaryTree(vector)

qreg = qiskit.QuantumRegister(2)
circ = qiskit.QuantumCircuit(qreg)

tree.preparation_circuit(circ, qreg)

print(tree)
print(circ)

### Loading the row norms of the matrix

### Loading the row vectors of the matrix

# More examples and advanced usage

### Example 4: Starting in a different state

In [ ]:
"""Example: Starting in a different state to estimate different singular values."""
matrix = np.random.randn(2, 2)
matrix += matrix.conj().T
qsve = QSVE(matrix)

classical = qsve.singular_values_classical()
print("Classically found singular values:", np.round(classical, 4))
top_from_zero = qsve.top_singular_values(nprecision_bits=7, ntop=4, singular_vector=None)
print("Top singular values starting from |0>:", np.round(top_from_zero, 4))
top_from_one = qsve.top_singular_values(nprecision_bits=7, ntop=4, singular_vector=[0, 1, 0, 0])
print("Top singular values starting from |1>:", np.round(top_from_one, 4))

You should see that different `singular_vector`s result in estimating different singular values with higher probability. (Recall: The top singular vectors returned from `QSVE.top_singular_values` are ordered by most probable to least probable.)

### Example 5: "Large" and sparse matrices

# Further Reading

TODO: Link to linear systems using QSVE and recommendation systems notebooks.

# References and notes

[1] Recommendation systems

[2] Linear systems QSVE

[3] There are $n^2$ eigenvalues of $W$, but there are only $n$ singular values of $A$. By symmetry of cosine about zero, exactly $2n$ eigenvalues of $W$ correspond to singular values of $A$. This means that there are $n^2 - 2n$ eigenvalues of $W$ _not_ related to singular values of $A$. What are these other eigenvalues? It can be easily seen that they are all one. While this does not affect the correctness of QSVE, it skews the measurement outcomes to contain a lot of all zero outcomes. This may be undesirable for large matrices. For the small examples we consider, however, this is not an issue.